<a href="https://colab.research.google.com/github/darshanvjani/ERA_vision_nlp_ai/blob/main/Backpropagation%20%26%20Architectural%20Basics/Session_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [28]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.conv1block = nn.Sequential(
        nn.Conv2d(1, 16, 3, padding = 0, bias = False),   # 28 -> 26 | 1 -> 3 | j = 1
        nn.ReLU(),
        nn.BatchNorm2d(16),
        nn.Dropout2d(0.02)
    )
    self.conv2block = nn.Sequential(
        nn.Conv2d(16, 16, 3, padding = 0, bias = False),   # 26 -> 24 | 3 -> 5 | j = 1
        nn.ReLU(),
        nn.BatchNorm2d(16),
        nn.Dropout2d(0.02)
    )
    self.conv3block = nn.Sequential(
        nn.Conv2d(16, 32, 3, padding = 0, bias = False),   # 24 -> 22 | 5 -> 7 | j = 1
        nn.ReLU(),
        nn.BatchNorm2d(32),
        nn.Dropout2d(0.02)
    )

    self.trans1block = nn.Sequential(
        nn.MaxPool2d(2,2),
        nn.Conv2d(32, 16, 1, padding = 0, bias = False),   # 22 -> 11 -> 11 | 5 -> 6 -> 6 | j = 1 -> 2 -> 2
        nn.ReLU()
    )

    self.conv4block = nn.Sequential(
        nn.Conv2d(16, 16, 3, padding = 0, bias = False),   # 11 -> 9 | 6 -> 10 | j = 2 -> 2
        nn.ReLU(),
        nn.BatchNorm2d(16),
        nn.Dropout2d(0.02)
    )
    self.conv5block = nn.Sequential(
        nn.Conv2d(16, 16, 3, padding = 0, bias = False),   # 9 -> 7 | 10 -> 14 | j = 2 -> 2
        nn.ReLU(),
        nn.BatchNorm2d(16),
        nn.Dropout2d(0.02)
    )

    self.conv6block = nn.Sequential(
        nn.Conv2d(16, 32, 3, padding = 0, bias = False),   # 7 -> 5 | 14 -> 18 | j = 2 -> 2
        nn.ReLU(),
        nn.BatchNorm2d(32),
        nn.Dropout2d(0.02)
    )

    self.outputblock = nn.Sequential(
        nn.AvgPool2d(5,5),
        nn.Conv2d(32, 10, 1, padding = 0, bias = False),   # 5 -> 1 -> 1 | 18 -> 26 -> 26 | j = 2 -> 2 -> 2

    )

  def forward(self, x):
          x = self.conv1block(x)
          x = self.conv2block(x)
          x = self.conv3block(x)
          # x = self.pool1(x)
          x = self.trans1block(x)
          x = self.conv4block(x)
          x = self.conv5block(x)
          x = self.conv6block(x)
          x = self.outputblock(x)

          x = x.view(-1, 10)
          return F.log_softmax(x, dim=-1)


In [2]:
# class Net(nn.Module):
#     def __init__(self):
#         super(Net, self).__init__()
#         self.conv1 = nn.Conv2d(1, 32, 3, padding=1) #input -? OUtput? RF
#         self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
#         self.pool1 = nn.MaxPool2d(2, 2)
#         self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
#         self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
#         self.pool2 = nn.MaxPool2d(2, 2)
#         self.conv5 = nn.Conv2d(256, 512, 3)
#         self.conv6 = nn.Conv2d(512, 1024, 3)
#         self.conv7 = nn.Conv2d(1024, 10, 3)

#     def forward(self, x):
#         x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
#         x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
#         x = F.relu(self.conv6(F.relu(self.conv5(x))))
#         x = F.relu(self.conv7(x))
#         x = x.view(-1, 10)
#         return F.log_softmax(x)

In [29]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             144
              ReLU-2           [-1, 16, 26, 26]               0
       BatchNorm2d-3           [-1, 16, 26, 26]              32
         Dropout2d-4           [-1, 16, 26, 26]               0
            Conv2d-5           [-1, 16, 24, 24]           2,304
              ReLU-6           [-1, 16, 24, 24]               0
       BatchNorm2d-7           [-1, 16, 24, 24]              32
         Dropout2d-8           [-1, 16, 24, 24]               0
            Conv2d-9           [-1, 32, 22, 22]           4,608
             ReLU-10           [-1, 32, 22, 22]               0
      BatchNorm2d-11           [-1, 32, 22, 22]              64
        Dropout2d-12           [-1, 32, 22, 22]               0
        MaxPool2d-13           [-1, 32, 11, 11]               0
           Conv2d-14           [-1, 16,

In [17]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [13]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [30]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.009, momentum=0.9)

for epoch in range(20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

loss=0.11600475758314133 batch_id=468: 100%|██████████| 469/469 [00:22<00:00, 20.85it/s]



Test set: Average loss: 0.0633, Accuracy: 9840/10000 (98%)



loss=0.10286415368318558 batch_id=468: 100%|██████████| 469/469 [00:20<00:00, 22.88it/s]



Test set: Average loss: 0.0373, Accuracy: 9888/10000 (99%)



loss=0.1184103712439537 batch_id=468: 100%|██████████| 469/469 [00:20<00:00, 22.82it/s]


...


Test set: Average loss: 0.0180, Accuracy: 9946/10000 (99%)



loss=0.007834059186279774 batch_id=468: 100%|██████████| 469/469 [00:20<00:00, 22.56it/s]



Test set: Average loss: 0.0197, Accuracy: 9939/10000 (99%)



loss=0.013240169733762741 batch_id=468: 100%|██████████| 469/469 [00:20<00:00, 22.54it/s]



Test set: Average loss: 0.0196, Accuracy: 9944/10000 (99%)

